In [8]:
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as sns
import pandas as pd
from random import random
import json, os.path, pickle

#result = pd.read_csv(f"test10.csv")
result = pd.read_csv(f"test3.csv")
result.columns

Index(['Unnamed: 0', 'student_set', 'comparision', 'students_per_group',
       'attempts_per_student', 'groupA', 'groupB', 'total_attempts', 'trials',
       'olswrong', 'olsns', 'olssig', 'olsvsig', 'olsp', 'geeBwrong', 'geeBns',
       'geeBsig', 'geeBvsig', 'geeBp', 'geeBIswrong', 'geeBIsns', 'geeBIssig',
       'geeBIsvsig', 'geeBIsp', 'query', 'time', 'key'],
      dtype='object')

In [9]:
print("Finding powered combinations where count of NS trials is less than 80%")
query = "comparision != 'none' and (" + ' or '.join([f"`{c}` < 20" for c in result.columns if c != 'olsns' and 'ns' in c])+")"
print(query)
result.query(query)

Finding powered combinations where count of NS trials is less than 80%
comparision != 'none' and (`geeBns` < 20 or `geeBIsns` < 20)


,Unnamed: 0,student_set,comparision,students_per_group,attempts_per_student,groupA,groupB,total_attempts,trials,olswrong,...,geeBvsig,geeBp,geeBIswrong,geeBIsns,geeBIssig,geeBIsvsig,geeBIsp,query,time,key
1,1,variable,vsmall,5,5,control,interventionA,500,3,0,...,1,0.504730,0,1,1,1,0.505742,"group == ['control', 'interventionA'] and pint...",1.626845e+12,b.3vsmall.5.5
2,2,variable,small,5,5,control,interventionB,500,3,0,...,0,0.685283,0,3,0,0,0.681148,"group == ['control', 'interventionB'] and pint...",1.626845e+12,b.3small.5.5
3,3,variable,mod,5,5,control,interventionBa,500,3,0,...,0,0.837152,0,1,2,0,0.833191,"group == ['control', 'interventionBa'] and pin...",1.626845e+12,b.3mod.5.5
4,4,variable,big,5,5,control,interventionC,500,3,0,...,1,0.500459,0,2,0,1,0.502176,"group == ['control', 'interventionC'] and pint...",1.626845e+12,b.3big.5.5
5,5,variable,huge,5,5,control,interventionD,500,3,0,...,0,0.402079,0,2,1,0,0.401132,"group == ['control', 'interventionD'] and pint...",1.626845e+12,b.3huge.5.5
6,6,variable,upper,5,5,interventionC,interventionD,500,3,0,...,0,1.547773,0,3,0,0,1.548864,"group == ['interventionC', 'interventionD'] an...",1.626845e+12,b.3upper.5.5
8,1,uniform,vsmall,5,5,control,interventionA,500,3,0,...,1,0.504730,0,1,1,1,0.505742,"group == ['control', 'interventionA'] and pint...",1.626845e+12,b.3vsmall.5.5
9,2,uniform,small,5,5,control,interventionB,500,3,0,...,0,0.685283,0,3,0,0,0.681148,"group == ['control', 'interventionB'] and pint...",1.626845e+12,b.3small.5.5
10,3,uniform,mod,5,5,control,interventionBa,500,3,0,...,0,0.837152,0,1,2,0,0.833191,"group == ['control', 'interventionBa'] and pin...",1.626845e+12,b.3mod.5.5
11,4,uniform,big,5,5,control,interventionC,500,3,0,...,1,0.500459,0,2,0,1,0.502176,"group == ['control', 'interventionC'] and pint...",1.626845e+12,b.3big.5.5


In [10]:
# Look for invalid measures:
print("These counts chould be ~2.5")
result.query("comparision == 'none'")[[c for c in result.columns if 'wrong' in c]]


These counts chould be ~2.5


,olswrong,geeBwrong,geeBIswrong
0,0,0,0
7,0,0,0
14,0,0,0
21,0,0,0


In [11]:
print("Finding powered combinations where count of NS trials is less than 80%")
query = "comparision != 'none' and (" + ' or '.join([f"{c} < 20" for c in result.columns if c != 'olsns' and 'ns' in c])+")"
result.query(query)

Finding powered combinations where count of NS trials is less than 80%


,Unnamed: 0,student_set,comparision,students_per_group,attempts_per_student,groupA,groupB,total_attempts,trials,olswrong,...,geeBvsig,geeBp,geeBIswrong,geeBIsns,geeBIssig,geeBIsvsig,geeBIsp,query,time,key
1,1,variable,vsmall,5,5,control,interventionA,500,3,0,...,1,0.504730,0,1,1,1,0.505742,"group == ['control', 'interventionA'] and pint...",1.626845e+12,b.3vsmall.5.5
2,2,variable,small,5,5,control,interventionB,500,3,0,...,0,0.685283,0,3,0,0,0.681148,"group == ['control', 'interventionB'] and pint...",1.626845e+12,b.3small.5.5
3,3,variable,mod,5,5,control,interventionBa,500,3,0,...,0,0.837152,0,1,2,0,0.833191,"group == ['control', 'interventionBa'] and pin...",1.626845e+12,b.3mod.5.5
4,4,variable,big,5,5,control,interventionC,500,3,0,...,1,0.500459,0,2,0,1,0.502176,"group == ['control', 'interventionC'] and pint...",1.626845e+12,b.3big.5.5
5,5,variable,huge,5,5,control,interventionD,500,3,0,...,0,0.402079,0,2,1,0,0.401132,"group == ['control', 'interventionD'] and pint...",1.626845e+12,b.3huge.5.5
6,6,variable,upper,5,5,interventionC,interventionD,500,3,0,...,0,1.547773,0,3,0,0,1.548864,"group == ['interventionC', 'interventionD'] an...",1.626845e+12,b.3upper.5.5
8,1,uniform,vsmall,5,5,control,interventionA,500,3,0,...,1,0.504730,0,1,1,1,0.505742,"group == ['control', 'interventionA'] and pint...",1.626845e+12,b.3vsmall.5.5
9,2,uniform,small,5,5,control,interventionB,500,3,0,...,0,0.685283,0,3,0,0,0.681148,"group == ['control', 'interventionB'] and pint...",1.626845e+12,b.3small.5.5
10,3,uniform,mod,5,5,control,interventionBa,500,3,0,...,0,0.837152,0,1,2,0,0.833191,"group == ['control', 'interventionBa'] and pin...",1.626845e+12,b.3mod.5.5
11,4,uniform,big,5,5,control,interventionC,500,3,0,...,1,0.500459,0,2,0,1,0.502176,"group == ['control', 'interventionC'] and pint...",1.626845e+12,b.3big.5.5
